In [1]:
#Shows each individual plot along with which model best fits it
import pickle, math
import pandas as pd
import sqlite3
import random
import os
#exec(open("./Imports.py").read())
execfile('Imports.py')
# _add_modules_path()
import Modules.Funcs as funcs
from Modules.Classes import Simulation
from Modules.Classes import CopyTweak
from Modules.Classes import CopyTweakRep
from Modules.Classes import Packer
from Modules.Classes import PackerRep
from Modules.Classes import ConjugateJK13
from Modules.Classes import RepresentJK13
from scipy.stats import stats as ss
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

# Specify default dataname
dbname = 'experiments-5con.db'#raw data
dataname_def = '5con'#bestparms also comes from here

#plotting options
STAT_LIMS =  (-1.0, 1.0)
#Fit to only last trial?
fitlast = False
#Make plots?
doplots = True
saveplots = False
if fitlast:
    showlast = True
else:
    showlast = False #True = Show last even if fitting to all?
#Bootstrap parameters
nbootstraps = 1000
if fitlast and showlast:
    savedir = 'private/individual_modelcomp_{}/fitlast'.format(dataname_def)
elif showlast:
    savedir = 'private/individual_modelcomp_{}/showonlylast/'.format(dataname_def)
elif fitlast:
    savedir = 'private/individual_modelcomp_{}/fitlast'.format(dataname_def)
else:
    savedir = 'private/individual_modelcomp_{}/showall'.format(dataname_def)
# savefilename='modelvsppt{}_t.pdf'.format(corrtype)
# modeleaseDB = "pickles/modelease_corr{}.p".format(corrtype)

#Some plotting options
font = {'family' : 'DejaVu Sans',
        'weight' : 'regular',
        'size'   : 15}

#Specify simulation values
N_SAMPLES = 10000
WT_THETA = 1.5
MIN_LL = 1e-10


participant_def = 'all'
unique_trials_def = 'all'
dataname = dataname_def
execfile('validate_data.py')

bestparmdb = "pickles/chtc_gs_best_params_{}".format(src)


plt.rc('font', **font)

# get data from pickle
with open(pickledir+src, "rb" ) as f:
    trials = pickle.load( f )

# get best params pickle
#bestparmdb = "pickles/chtc_gs_best_params_all_data_e1_e2.p"
#bestparmdb = "pickles/chtc_gs_best_params_corrs.p"
with open(bestparmdb, "rb" ) as f:
    best_params_t = pickle.load( f )

#Rebuild it into a smaller dict
best_params = dict()
for modelname in best_params_t.keys():    
    best_params[modelname] = dict()
    for i,parmname in enumerate(best_params_t[modelname]['parmnames']):
        parmval = best_params_t[modelname]['bestparmsll']
        best_params[modelname][parmname] = parmval[i]
modelList = [Packer,RepresentJK13]
#modelList = [CopyTweak,CopyTweakRep,Packer, RepresentJK13,]                            

#Specify plot order
modelPlotOrder = np.array([[Packer,RepresentJK13],[CopyTweak,ConjugateJK13]])
#modelPlotOrder = np.array([[CopyTweak,CopyTweakRep],[Packer,RepresentJK13]])

#Prepare matched database    
matchdb='../cat-assign/data_utilities/cmp_midbot.db'
        
unique_trials = 'all'
trials.task = task




In [2]:
con = sqlite3.connect(dbname)
participants = pd.read_sql_query("SELECT participant, condition from participants", con)
generation = pd.read_sql_query("SELECT * from generation", con)
alphas = pd.read_sql_query("SELECT * from alphas", con)
stimuli = pd.read_sql_query("SELECT * from stimuli", con).values
con.close()


In [3]:
#Go through each participant and get their fits for each model
ll = []
ll_trials = []
tally = np.zeros(2)
wins = []
for ppt in [318]:#trials.participants:
    ppt = int(ppt)
    print('Ppt: ' + str(ppt))
    llrow = []
    if fitlast:
        extract_trials = 3
    else:
        extract_trials = 'all'
    conditionStr = participants.condition[participants.participant==ppt].values
    pptconstr = conditionStr[0][0:2]
    if "Corner" in conditionStr[0]:
        pptconstr += conditionStr[0][-1] #Add info on whether square or circle
    
    trialppt = Simulation.extractPptData(trials, ppt=ppt, unique_trials=extract_trials)
    #Get list of lls and winning model
    for model in modelList:
        params = best_params[model.model]
        ppt_trial_ll = trialppt.loglike(params=params,model=model,parmxform=False)
        llrow += [ppt_trial_ll]
    bestmodel = llrow.index(np.min(llrow))
    bestmodelstr = modelList[bestmodel].modelshort
    ll += [[ppt]+llrow]
    tally[bestmodel] += 1
    wins += [bestmodel]
    #Prepare stuff fot plotting
    if doplots:
        
        #Sort trial obj by trial number - really only applicable if onlylast is false                                                                                                                   
        trialOrder = []
        maxbeta = 0
        for t,trialobj in enumerate(trialppt.Set):
            nbeta = len(trialobj['categories'][1])
            trialppt.Set[t]['trial'] = nbeta
            trialOrder += [nbeta]
            maxbeta = max(nbeta,maxbeta)
        trialOrder = np.argsort(trialOrder)
        wrap_ax = trialppt.Set[0]['wrap_ax'][0]
        if wrap_ax==1:
            wrapstr = '<-Wrapped->'
        else:
            wrapstr = ''
        #Add dummy last trial to plot gradients for hypothetical fifth beta
        newcatB = np.append(trialppt.Set[trialOrder[maxbeta]]['categories'][1],trialppt.Set[trialOrder[maxbeta]]['response'])
        newcatAll = [trialppt.Set[trialOrder[maxbeta]]['categories'][0],newcatB]
        trialppt.add(response=0,categories=newcatAll, participant = ppt,wrap_ax = wrap_ax)
        trialOrder = np.append(trialOrder,maxbeta+1)
        
        if showlast:
            ntrials = 1
            plottrials = [len(trialppt.Set)-1] #plot only last trial
        else:
            ntrials = len(trialppt.Set)
            plottrials = range(ntrials)
        f,ax = plt.subplots(ntrials,len(modelList),figsize = (8, 2*ntrials+2.5))
        for trial in plottrials:
            categories = [trialppt.stimuli[i,:] for i in trialppt.Set[trialOrder[trial]]['categories'] if len(i)>0]
            A = categories[0]
            resp = trialppt.stimuli[trialppt.Set[trialOrder[trial]]['response'],:]
            if len(categories)>1:
                #include the response                                                                                                                  
                B = np.append(categories[1],resp,axis=0)
                if showlast:
                    #Include response in generatin of ps                
                    categories[1] = B
            else:
                #categories += [resp]
                B = resp            
            ps = []
            ll_trial = []
            for m,model in enumerate(modelList):
                if m==bestmodel:
                    winstr = '*'
                else:
                    winstr = ''
                if ntrials>1:
                    ax[0,m].set_title('{}{}{}, ll={:.2f}'.format(winstr,model.modelshort,winstr,llrow[m]))
                else:                    
                    ax[m].set_title('{}{}{}, ll={:.2f}'.format(winstr,model.modelshort,winstr,llrow[m]))
                params = best_params[model.model]
                #Plot heatmap for each model
                ps += [model(categories,params,trialppt.stimrange).get_generation_ps(trialppt.stimuli,1,'generate',wrap_ax=wrap_ax)]
                #Get lls for each trial step
                ll_trial += [trialppt.loglike(params=params,model=model,parmxform=False,whole_array=True)]

            #Plot the individual plots
            plotct = 0
            plotVals = []
            psMax = 0
            psMin = 1
            #Get range                                                                                                                                     
            for ps_el in ps:
                psMax = max(psMax,ps_el.max())
                psMin = min(psMin,ps_el.min())

            #Normalise all values                                                                                                                          
            psRange = psMax-psMin
            for i,ps_el in enumerate(ps): #each ps element correspond to a model
                plotct += 1
                gps = funcs.gradientroll(ps_el,'roll')[:,:,0]
                ps_ElRange = gps.max()-gps.min();
                plotVals += [(gps-gps.min())/ps_ElRange]                                                                             
                betacol = ['green' for bi in range(len(B))]
#                 if task is 'generate':
#                     betacol[len(B)-1] = 'orange'
                if ntrials>1:
                    betacol[len(B)-1] = 'orange'
                    if trial == ntrials-1:
                        betacol[len(B)-1] = [0,0,0,0]
                    im = funcs.plotgradient(ax[trial,i], plotVals[i], A, B, clim = STAT_LIMS, cmap = 'PuOr',beta_col=betacol)
                    ax[trial,i].set_ylabel('Trial {}\n{}'.format(trial,wrapstr))
                    if trial<ntrials-1:
                        ax[trial,i].set_xlabel('ll_trial = {:.2f}'.format(ll_trial[i][trial]),fontsize=10)
                    #else:
                        
                else:
                    im = funcs.plotgradient(ax[i], plotVals[i], A, B, clim = STAT_LIMS, cmap = 'PuOr',beta_col=betacol)
#                     ax[i].set_ylabel('Trial {}'.format(trial))
        #Save fig
        if saveplots:
            savedirext = '{}/{}'.format(savedir,bestmodelstr)
            if os.path.isdir(savedirext):
                plt.savefig('{}/{}_{}_{}'.format(savedirext,pptconstr,int(ppt),bestmodelstr[0]))
            else:
                os.mkdir(savedirext)
                plt.savefig('{}/{}_{}_{}'.format(savedirext,pptconstr,int(ppt),bestmodelstr[0]))
        plt.close()
        ll_trials += [np.array(ll_trial).T]


    


Ppt: 318


In [4]:
best_params['Hierarchical Sampling With Representativeness']

{'category_mean_bias': 5.911656375965484,
 'category_variance_bias': 1.0000000001002045,
 'determinism': 1.862840518192873,
 'domain_variance_bias': 1.4544815236039712}

In [5]:
# best_params['Hierarchical Sampling With Representativeness']
# temp_params = best_params['Hierarchical Sampling With Representativeness'].copy()
# temp_params['domain_variance_bias'] = .0001
# categories1 = [np.array([[-1,-1],[1,-1],[-1,1],[1,1]])]
# wrap_ax = None

# temp1 = model(categories1,temp_params,trialppt.stimrange).get_generation_ps(trialppt.stimuli,1,'generate',wrap_ax=wrap_ax)
# print(np.round(np.reshape(temp1,(9,9)),3))

[[0.036 0.023 0.016 0.013 0.013 0.013 0.016 0.023 0.036]
 [0.023 0.014 0.01  0.009 0.008 0.009 0.01  0.014 0.023]
 [0.016 0.01  0.007 0.006 0.006 0.006 0.007 0.01  0.016]
 [0.013 0.009 0.006 0.005 0.005 0.005 0.006 0.009 0.013]
 [0.013 0.008 0.006 0.005 0.004 0.005 0.006 0.008 0.013]
 [0.013 0.009 0.006 0.005 0.005 0.005 0.006 0.009 0.013]
 [0.016 0.01  0.007 0.006 0.006 0.006 0.007 0.01  0.016]
 [0.023 0.014 0.01  0.009 0.008 0.009 0.01  0.014 0.023]
 [0.036 0.023 0.016 0.013 0.013 0.013 0.016 0.023 0.036]]


In [6]:
#HOW COME REP FAILS THE TRANSLATION TEST?!
model = RepresentJK13
params = best_params[model.model]
categories1 = [np.array([[-1,-1],[1,-1],[-1,1],[1,1]])]
temp1 = model(categories1,params,trialppt.stimrange).get_generation_ps(trialppt.stimuli,1,'generate',wrap_ax=wrap_ax)
print(np.round(np.reshape(temp1,(9,9)),3))
categories2 = [np.array([[-1,-1],[1,-1],[-1,-.75],[1,-.75]])]
temp2 = model(categories2,params,trialppt.stimrange).get_generation_ps(trialppt.stimuli,1,'generate',wrap_ax=wrap_ax)
print(np.round(np.reshape(temp2,(9,9)),3))

[[0.029 0.02  0.016 0.014 0.013 0.014 0.016 0.02  0.029]
 [0.02  0.014 0.011 0.01  0.009 0.01  0.011 0.014 0.02 ]
 [0.016 0.011 0.009 0.007 0.007 0.007 0.009 0.011 0.016]
 [0.014 0.01  0.007 0.006 0.006 0.006 0.007 0.01  0.014]
 [0.013 0.009 0.007 0.006 0.006 0.006 0.007 0.009 0.013]
 [0.014 0.01  0.007 0.006 0.006 0.006 0.007 0.01  0.014]
 [0.016 0.011 0.009 0.007 0.007 0.007 0.009 0.011 0.016]
 [0.02  0.014 0.011 0.01  0.009 0.01  0.011 0.014 0.02 ]
 [0.029 0.02  0.016 0.014 0.013 0.014 0.016 0.02  0.029]]
[[0.004 0.003 0.002 0.002 0.002 0.002 0.002 0.003 0.004]
 [0.003 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.003]
 [0.003 0.002 0.002 0.001 0.001 0.001 0.002 0.002 0.003]
 [0.003 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.003]
 [0.004 0.003 0.002 0.002 0.002 0.002 0.002 0.003 0.004]
 [0.007 0.005 0.004 0.003 0.003 0.003 0.004 0.005 0.007]
 [0.014 0.01  0.008 0.007 0.006 0.007 0.008 0.01  0.014]
 [0.033 0.024 0.018 0.016 0.015 0.016 0.018 0.024 0.033]
 [0.096 0.068 0.053 0.045 0.04

In [7]:
##Probabilities at outer edge vs inner space - Rep
outind = range(9) + [(e+1)*9 for e in range(7)] + [(e+1)*9+8 for e in range(7)] + range(72,80)
innind = [i for i in range(81) if not i in outind]
model = RepresentJK13
params = best_params[model.model]
categories1 = [np.array([[-1,-1],[1,-1],[-1,1],[1,1]])]
temp1 = model(categories1,params,trialppt.stimrange).get_generation_ps(trialppt.stimuli,1,'generate',wrap_ax=wrap_ax)
outps = sum(temp1[outind])
innps = sum(temp1[innind])
print(np.round(np.reshape(temp1,(9,9)),3))

print('Outer edges: ' + str(np.round(outps,2)))
print('Inner spaces: ' + str(np.round(innps,2)))

[[0.029 0.02  0.016 0.014 0.013 0.014 0.016 0.02  0.029]
 [0.02  0.014 0.011 0.01  0.009 0.01  0.011 0.014 0.02 ]
 [0.016 0.011 0.009 0.007 0.007 0.007 0.009 0.011 0.016]
 [0.014 0.01  0.007 0.006 0.006 0.006 0.007 0.01  0.014]
 [0.013 0.009 0.007 0.006 0.006 0.006 0.007 0.009 0.013]
 [0.014 0.01  0.007 0.006 0.006 0.006 0.007 0.01  0.014]
 [0.016 0.011 0.009 0.007 0.007 0.007 0.009 0.011 0.016]
 [0.02  0.014 0.011 0.01  0.009 0.01  0.011 0.014 0.02 ]
 [0.029 0.02  0.016 0.014 0.013 0.014 0.016 0.02  0.029]]
Outer edges: 0.53
Inner spaces: 0.47


In [9]:
##Probabilities at outer edge vs inner space - Packer
outind = range(9) + [(e+1)*9 for e in range(7)] + [(e+1)*9+8 for e in range(7)] + range(72,80)
innind = [i for i in range(81) if not i in outind]
model = Packer
params = best_params[model.model]
categories1 = [np.array([[-1,-1],[1,-1],[-1,1],[1,1]])]
temp1 = model(categories1,params,trialppt.stimrange).get_generation_ps(trialppt.stimuli,1,'generate',wrap_ax=wrap_ax)
outps = sum(temp1[outind])
innps = sum(temp1[innind])
print(np.round(np.reshape(temp1,(9,9)),3))

print('Outer edges: ' + str(np.round(outps,2)))
print('Inner spaces: ' + str(np.round(innps,2)))

[[0.009 0.01  0.011 0.011 0.012 0.011 0.011 0.01  0.009]
 [0.01  0.011 0.012 0.013 0.013 0.013 0.012 0.011 0.01 ]
 [0.011 0.012 0.013 0.014 0.014 0.014 0.013 0.012 0.011]
 [0.011 0.013 0.014 0.015 0.015 0.015 0.014 0.013 0.011]
 [0.012 0.013 0.014 0.015 0.015 0.015 0.014 0.013 0.012]
 [0.011 0.013 0.014 0.015 0.015 0.015 0.014 0.013 0.011]
 [0.011 0.012 0.013 0.014 0.014 0.014 0.013 0.012 0.011]
 [0.01  0.011 0.012 0.013 0.013 0.013 0.012 0.011 0.01 ]
 [0.009 0.01  0.011 0.011 0.012 0.011 0.011 0.01  0.009]]
Outer edges: 0.33
Inner spaces: 0.67


In [ ]:
ppt_S = gencond.loc[gencond.trial==0].loc[gencond.condition=='Corner_S'].stimulus.values
ppt_C = gencond.loc[gencond.trial==0].loc[gencond.condition=='Corner_C'].stimulus.values
outppt = [i for i in ppt_S if i in outind]
outppt


In [12]:
#Count frequencies of people 
gencond = pd.merge(generation,participants,on='participant')
outind = range(9) + [(e+1)*9 for e in range(7)] + [(e+1)*9+8 for e in range(7)] + range(72,80)
innind = [i for i in range(81) if not i in outind]
#Get people
# ppt_S = gencond.loc[gencond.trial==0].loc[gencond.condition=='Corner_S'].stimulus.values
ppt_C = gencond.loc[gencond.trial==0].loc[gencond.condition=='Corner_C'].stimulus.values
outppt = [i for i in ppt_C if i in outind]# + [i for i in ppt_C if i in outind]
innppt = [i for i in ppt_C if i in innind]# + [i for i in ppt_C if i in innind]

print('Ppts 1st exemplar at circle edges: ' + str(len(outppt)))
print('Ppts 1st exemplar at circle inner spaces: ' + str(len(innppt)))

outppt = [i for i in ppt_S if i in outind]# + [i for i in ppt_C if i in outind]
innppt = [i for i in ppt_S if i in innind]# + [i for i in ppt_C if i in innind]

print('Ppts 1st exemplar at square edges: ' + str(len(outppt)))
print('Ppts 1st exemplar at square inner spaces: ' + str(len(innppt)))

Ppts 1st exemplar at circle edges: 5
Ppts 1st exemplar at circle inner spaces: 38
Ppts 1st exemplar at square edges: 16
Ppts 1st exemplar at square inner spaces: 30


In [ ]:
#Checkfail comes from the ind_dummy notebook where I look for ppts whose winning order doesn't match the majority of wins across all permutations 
#That is to say, the order of the generated betas seem to matter most for these guys
checkfail = np.asarray([11.0, 14.0, 15.0, 56.0, 62.0, 66.0, 70.0, 98.0, 132.0, 133.0, 141.0, 152.0, 160.0, 163.0, 170.0, 171.0, 178.0, 186.0, 187.0, 201.0, 203.0, 210.0, 213.0, 215.0, 216.0, 221.0, 225.0, 226.0, 232.0, 239.0, 267.0, 273.0, 275.0, 300.0],dtype=int)
ll_array = np.array(ll)
ll_checkfail = ll_array[checkfail,:]
ll_checkfail

In [ ]:
trialppt.Set

In [ ]:
np.diff(np.mean(np.array(ll_trials),axis=0),axis=1) #positive = packer wins

In [ ]:
#Calculate cumulative lls at each trial, excluding last trial (since it's a dummy)
ll_trials_cs = [np.cumsum(ppt_trial[:4,:],axis=0) for ppt_trial in ll_trials]
#Get wins at each trial
ll_trials_mins = [np.argmin(ppt_trial[:4,:],axis=1) for ppt_trial in ll_trials_cs]
ll_trials_means = np.mean(np.array(ll_trials),axis=0)
repwins = np.sum(np.array(ll_trials_mins),axis=0)
packerwins = len(ll_trials) - repwins
print('Packer wins, mean lls across trials:')
print(packerwins)
print
print('Representativeness wins, mean lls across trials:')
print(repwins)

In [ ]:
#Print a summary
print('Tally of bests:')

for m,model in enumerate(modelList):
    print('%s: %.2f' % (model.modelshort,tally[m]))

In [ ]:
trialppt.Set

In [ ]:
trialppt.Set[len(trialppt.Set)-1]['categories']

In [ ]:
temp= cp.deepcopy(trialppt)
newcatB = np.append(temp.Set[len(trialppt.Set)-1]['categories'][1],temp.Set[len(trialppt.Set)-1]['response'])
newcatAll = [temp.Set[len(trialppt.Set)-1]['categories'][0],newcatB]
temp.add(response=np.array([0,0]),categories=newcatAll, participant = ppt)

In [ ]:
trialppt.loglike(params=params,model=model,parmxform=False,whole_array=False)

In [ ]:
sum(trialppt.loglike(params=params,model=model,parmxform=False,whole_array=True))

In [ ]:
trialppt.loglike(params=params,model=model,parmxform=False,whole_array=True)

In [ ]:
newcatAll

In [ ]:
maxbeta

In [ ]:
trialOrder

In [ ]:
lla = np.array(ll)
packerwins = np.bool_(1-np.array(wins))
repwins = np.bool_(wins)
plist = np.abs(lla[packerwins,1]-lla[packerwins,2])
rlist = np.abs(lla[repwins,1]-lla[repwins,2])
print(np.mean(plist),np.min(plist),np.max(plist))
print(np.mean(rlist),np.min(rlist),np.max(rlist))
print(sum(lla[packerwins,1]))
print(sum(lla[repwins,2]))

In [ ]:
# f = plt.figure()
plt.hist(rlist,histtype='step')
plt.hist(plist,histtype='step')

In [ ]:
plt.hist(plist)

In [ ]:
len(rlist)